## Analyse 

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel("AFC.xlsx", sheet_name="TDC", index_col=0)
df

,Bleu,Blanc,Leger,Lourd,Tres lourd,carre
Produit,,,,,,
P1,1,0,0,1,0,0
P2,0,1,1,0,0,1
P3,0,0,1,0,0,0
P4,1,0,0,0,1,1
P5,0,1,1,0,0,0
P6,0,0,0,0,1,1
P7,1,0,1,0,0,0
P8,0,1,0,0,1,1
P9,0,0,0,0,1,0


**Remarque :** Pour le tableau de Burt, on ne tient pas cen compte de l'ordre établie pour la variable ordinale donnée.

### Tableau de Burt
C'est à partir de ce tableu que l'on va effectuer

In [4]:
T = df.to_numpy()
T_burt = np.dot(T.T, T)

In [5]:
T_burt

array([[4, 0, 1, 2, 1, 2],
       [0, 3, 2, 0, 1, 2],
       [1, 2, 4, 0, 0, 1],
       [2, 0, 0, 2, 0, 1],
       [1, 1, 0, 0, 4, 3],
       [2, 2, 1, 1, 3, 5]], dtype=int64)

In [6]:
# Convertir le résultat en DataFrame pour une meilleure lisibilité
result_df = pd.DataFrame(T_burt, columns=df.columns, index=df.columns)
result_df


,Bleu,Blanc,Leger,Lourd,Tres lourd,carre
Bleu,4,0,1,2,1,2
Blanc,0,3,2,0,1,2
Leger,1,2,4,0,0,1
Lourd,2,0,0,2,0,1
Tres lourd,1,1,0,0,4,3
carre,2,2,1,1,3,5


## Tableau des distances 

Cest le tableau par excellence qui permet, à travers une mesure de similarité ou de dissimilarité, de mesurer les distances de chaque paire d'individu. Il faut noté que la distance euclidienne ne peut pas s'appliquer dans ce contexte car les modalités ne sont pas mesurables mais plutôt des attributs.
Le choix dde la distance est dicté par l'apporche méthodologique choisie dans l'algorithme **IBM Master Plan**

In [3]:
df1 = pd.read_excel("AFC.xlsx", sheet_name="Feuil3", index_col=0)
df1

,Reponse 1,Unnamed: 2,Reponse 2,Unnamed: 4
NaN,V1,V2,V3,V4
I1,1,0,1,0
I2,1,0,0,1
I3,0,1,1,0
I4,0,1,0,1
I5,1,0,1,0
I6,1,0,0,1
I7,0,1,1,0


In [8]:
entete = ["V1", "V2", "V3", "V4"]
df1.columns = entete
df1 = df1.drop(index=df1.index[0])
df1

,V1,V2,V3,V4
I1,1,0,1,0
I2,1,0,0,1
I3,0,1,1,0
I4,0,1,0,1
I5,1,0,1,0
I6,1,0,0,1
I7,0,1,1,0


A partir de ce tableau, nous allons donner la matrice des distances correspondantes. On peut utiliserr la distance de dissimilarité :

$d(i,j) = b+c / a+b+c+d$ 

In [24]:
import numpy as np
import pandas as pd

T_sim = df1

T = T_sim.to_numpy()

# Calculer la matrice des distances de manière vectorisée
a = np.dot(T, T.T)  # (1,1) - produit scalaire des lignes
b = np.dot(1 - T, 1 - T.T)  # (0,0) - produit scalaire des négatifs des lignes
c = np.dot(T, (1 - T).T) + np.dot((1 - T), T.T)  # (1,0) + (0,1)

distance_matrix1 = (b + c) / (a + b + c)

distance_df1 = pd.DataFrame(distance_matrix1, index=df.index, columns=df.index)

np.fill_diagonal(distance_df1.values, 0)

print(distance_df1)

      I1    I2    I3    I4    I5    I6    I7
I1     0  0.75  0.75   1.0   0.5  0.75  0.75
I2  0.75     0   1.0  0.75  0.75   0.5   1.0
I3  0.75   1.0     0  0.75  0.75   1.0   0.5
I4   1.0  0.75  0.75     0   1.0  0.75  0.75
I5   0.5  0.75  0.75   1.0     0  0.75  0.75
I6  0.75   0.5   1.0  0.75  0.75     0   1.0
I7  0.75   1.0   0.5  0.75  0.75   1.0     0


In [22]:
import numpy as np
mat = np.zeros((7,7))
tdf = df

def distance(a, b, c, d):
    return np.array((b + c) / (a + b + c + d))


for i in range(len(tdf)):
    for j in range (len(tdf)):
        a=b=c=d=0
        if i <=j:
            mat[i][j]=0
            
        else:
            for k in range (4):
                
                if tdf.iloc[i,k]==tdf.iloc[j,k] and tdf.iloc[i,k]==1:
                    a+=1
                elif tdf.iloc[i,k]==tdf.iloc[j,k] and tdf.iloc[i,k]==0:
                    d+=1
                elif tdf.iloc[i,k]!=tdf.iloc[j,k] and tdf.iloc[i,k]==0:
                    b+=1
                elif tdf.iloc[i,k]!=tdf.iloc[j,k] and tdf.iloc[i,k]==1:
                    c+=1
            mat[i][j]=distance(a,b,c,d)

mat

array([[0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0.5, 1. , 0. , 0. , 0. , 0. , 0. ],
       [1. , 0.5, 0.5, 0. , 0. , 0. , 0. ],
       [0. , 0.5, 0.5, 1. , 0. , 0. , 0. ],
       [0.5, 0. , 1. , 0.5, 0.5, 0. , 0. ],
       [0.5, 1. , 0. , 0.5, 0.5, 1. , 0. ]])

In [9]:
import pandas as pd
import numpy as np

# Fonction pour calculer la matrice des distances
def calculate_distance_matrix(df):
    n = len(df)  # Nombre de lignes
    distance_matrix = np.zeros((n, n))  # Initialiser une matrice n x n

    for i in range(n):
        for j in range(i):
            if i != j:  # On ne calcule pas la distance pour la diagonale
                row_i = df.iloc[i, :]
                row_j = df.iloc[j, :]

                # Calcul de A, b, c, d
                A = sum((row_i == 1 ) & (row_j == 1))  # Co-occurrences positives
                b = sum((row_i == 1) & (row_j == 0))  # Présent dans i, absent dans j
                c = sum((row_i == 0) & (row_j == 1))  # Absent dans i, présent dans j
                d = sum((row_i == 0) & (row_j == 0))  # Absent dans les deux

                # Calcul de la distance avec la formule donnée
                distance_matrix[i, j] = (b + c) / (A + b + c + d)

    # Retourner un DataFrame pour plus de clarté
    return pd.DataFrame(distance_matrix, index=df.index, columns=df.index)

# Calcul de la matrice des distances
distance_matrix = calculate_distance_matrix(df)
distance_matrix

,I1,I2,I3,I4,I5,I6,I7
I1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I2,0.5,0.0,0.0,0.0,0.0,0.0,0.0
I3,0.5,1.0,0.0,0.0,0.0,0.0,0.0
I4,1.0,0.5,0.5,0.0,0.0,0.0,0.0
I5,0.0,0.5,0.5,1.0,0.0,0.0,0.0
I6,0.5,0.0,1.0,0.5,0.5,0.0,0.0
I7,0.5,1.0,0.0,0.5,0.5,1.0,0.0


In [18]:
# Extraire la partie inférieure diagonale
lower_triangle = np.tril(distance_matrix.values, k=-1)  # k=-1 pour exclure la diagonale
lower_triangle_df = pd.DataFrame(lower_triangle, index=distance_matrix.index, columns=distance_matrix.columns)
lower_triangle_df


,I1,I2,I3,I4,I5,I6,I7
I1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I2,0.5,0.0,0.0,0.0,0.0,0.0,0.0
I3,0.5,1.0,0.0,0.0,0.0,0.0,0.0
I4,1.0,0.5,0.5,0.0,0.0,0.0,0.0
I5,0.0,0.5,0.5,1.0,0.0,0.0,0.0
I6,0.5,0.0,1.0,0.5,0.5,0.0,0.0
I7,0.5,1.0,0.0,0.5,0.5,1.0,0.0
